In [ ]:
import duckdb, time

"""
This function performs the same aggregation as the Pandas version,
but uses DuckDB to query the Parquet file directly using SQL.

Benefits:
- Lower memory usage (due to vectorized and columnar processing)
- Direct query execution on the Parquet file without full in-memory load

Metrics collected:
- Execution time
"""

start = time.time()
con = duckdb.connect()

# SQL direto sobre Parquet
query = """
SELECT
    passenger_count,
    COUNT(*) AS total_rides,
    AVG(trip_distance) AS avg_distance,
    AVG(total_amount) AS avg_amount
FROM 'base/yellow_tripdata_2023-01.parquet'
GROUP BY passenger_count
ORDER BY passenger_count
"""
df_duck = con.execute(query).df()
print("DuckDB:", round(time.time() - start, 2), "s")
print(df_duck)


DuckDB: 0.14 s
    passenger_count  total_rides  avg_distance  avg_amount
0               0.0        51164      2.761904   24.162124
1               1.0      2261400      3.338169   26.443472
2               2.0       451536      3.931051   29.313282
3               3.0       106353      3.664393   28.475420
4               4.0        53745      3.812581   29.611602
5               5.0        42681      3.282478   26.588261
6               6.0        28124      3.250963   26.558484
7               7.0            6      4.238333   85.111667
8               8.0           13      4.270769   99.336923
9               9.0            1      0.000000   92.250000
10              NaN        71743     21.011154   29.133590


In [ ]:
import pandas as pd, time

  """
  This function loads a large Parquet file using Pandas and performs
  a group-by operation to aggregate data based on passenger count.

  Metrics collected:
  - Execution time
  - Memory usage (via memory_profiler)

  The function computes:
  - Total number of rides
  - Average trip distance
  - Average total amount per passenger count
  """

start = time.time()
df = pd.read_parquet("base/yellow_tripdata_2023-01.parquet")

agg = (
   df.groupby("passenger_count")
     .agg(
       total_rides=("passenger_count","count"),
       avg_distance=("trip_distance","mean"),
       avg_amount=("total_amount","mean")
     )
     .reset_index()
)

print("Pandas:", round(time.time() - start, 2), "s")
print(agg)


Pandas: 0.61 s
   passenger_count  total_rides  avg_distance  avg_amount
0              0.0        51164      2.761904   24.162124
1              1.0      2261400      3.338169   26.443472
2              2.0       451536      3.931051   29.313282
3              3.0       106353      3.664393   28.475420
4              4.0        53745      3.812581   29.611602
5              5.0        42681      3.282478   26.588261
6              6.0        28124      3.250963   26.558484
7              7.0            6      4.238333   85.111667
8              8.0           13      4.270769   99.336923
9              9.0            1      0.000000   92.250000
